In [2]:
pip install sparknlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.4/453.4 kB 1.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=01621bb9feb31b31903a3e7d4fcf23f8ae0f254e5babc49d0af246d3588d38df
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyspark
from pyspark import SparkContext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.types import StructType, StringType, IntegerType
from  pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import StopWordsRemover
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, CountVectorizer

In [4]:
TRAIN_PATH = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip"
TEXT = "comment_text"
STOP_SYMBOLS = '[0123456789»«.,—"\'-?:!;\n%@| "]'
TARGET = "toxic"
DROP_LIST = [ "id","severe_toxic", "obscene","threat","insult","identity_hate"]
WEIGHT_COL = "weight"

In [5]:
# getting english stop words
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
stop.append("")
for a in STOP_SYMBOLS:
    stop.append(str(a))

In [6]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
spark = SparkSession.builder \
                    .appName('Toxic Comment Classification') \
                    .config("spark.driver.maxResultSize","0") \
                    .config("spark.sql.crossJoin.enabled", "true") \
                    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer") \
                    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
                    .config("spark.driver.memory", "20g") \
                    .config("spark.sql.shuffle.partitions", "30") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/17 20:08:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
#used pandas because couldn't understand how to parse this poor quotes with spark
df = pd.read_csv(TRAIN_PATH)
df=spark.createDataFrame(df)

df = df.select([column for column in df.columns if column not in DROP_LIST])
df.show(5)

22/12/17 20:10:51 WARN TaskSetManager: Stage 0 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|        comment_text|toxic|
+--------------------+-----+
|Explanation\nWhy ...|    0|
|D'aww! He matches...|    0|
|Hey man, I'm real...|    0|
|"\nMore\nI can't ...|    0|
|You, sir, are my ...|    0|
+--------------------+-----+
only showing top 5 rows



In [10]:
import numpy as np
#creating class balance
y_collect = df.select(TARGET).groupBy(TARGET).count().collect()
unique_y = [x[TARGET] for x in y_collect]
total_y = sum([x["count"] for x in y_collect])
unique_y_count = len(y_collect)
bin_count = [x["count"] for x in y_collect]

class_weights_spark = {i: ii for i, ii in zip(unique_y, total_y / (unique_y_count * np.array(bin_count)))}
#very unbalaned dataframe
print(f" class weights are {class_weights_spark}") 

22/12/17 20:11:02 WARN TaskSetManager: Stage 1 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


 class weights are {0: 0.5530022110246262, 1: 5.216784359879691}


In [11]:
from itertools import chain
mapping_expr = F.create_map([F.lit(x) for x in chain(*class_weights_spark.items())])

df = df.withColumn("weight", mapping_expr.getItem(F.col(TARGET)))

/opt/conda/lib/python3.7/site-packages/pyspark/sql/column.py:423: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  FutureWarning,


In [12]:
df.show()

22/12/17 20:11:04 WARN TaskSetManager: Stage 4 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+-----+------------------+
|        comment_text|toxic|            weight|
+--------------------+-----+------------------+
|Explanation\nWhy ...|    0|0.5530022110246262|
|D'aww! He matches...|    0|0.5530022110246262|
|Hey man, I'm real...|    0|0.5530022110246262|
|"\nMore\nI can't ...|    0|0.5530022110246262|
|You, sir, are my ...|    0|0.5530022110246262|
|"\n\nCongratulati...|    0|0.5530022110246262|
|COCKSUCKER BEFORE...|    1| 5.216784359879691|
|Your vandalism to...|    0|0.5530022110246262|
|Sorry if the word...|    0|0.5530022110246262|
|alignment on this...|    0|0.5530022110246262|
|"\nFair use ratio...|    0|0.5530022110246262|
|bbq \n\nbe a man ...|    0|0.5530022110246262|
|Hey... what is it...|    1| 5.216784359879691|
|Before you start ...|    0|0.5530022110246262|
|Oh, and the girl ...|    0|0.553002211024

In [13]:
#tokenization
tokenizer = Tokenizer(inputCol=TEXT,outputCol="text_tokenized") 
#stop words removing
remover = StopWordsRemover(stopWords=stop,inputCol= tokenizer.getOutputCol(), outputCol="result")
hashingTF = HashingTF(inputCol = remover.getOutputCol(), outputCol="rawFeatures",numFeatures=10000)
idf = IDF(inputCol= hashingTF.getOutputCol(), outputCol="features", minDocFreq=5)
label_stringIdx = StringIndexer(inputCol = TARGET, outputCol = "label")
#idfModel = idf.fit(texts)
#rescaledData = idfModel.transform(texts)

In [14]:
pipeline = Pipeline() \
.setStages([tokenizer,remover, hashingTF, idf, label_stringIdx])

In [15]:
pipelineFit = pipeline.fit(df)
dataset = pipelineFit.transform(df)
dataset.show()

22/12/17 20:11:05 WARN TaskSetManager: Stage 5 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:11:15 WARN TaskSetManager: Stage 6 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:11:16 WARN TaskSetManager: Stage 9 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+-----+------------------+--------------------+--------------------+--------------------+--------------------+-----+
|        comment_text|toxic|            weight|      text_tokenized|              result|         rawFeatures|            features|label|
+--------------------+-----+------------------+--------------------+--------------------+--------------------+--------------------+-----+
|Explanation\nWhy ...|    0|0.5530022110246262|[explanation, why...|[explanation, edi...|(10000,[360,515,5...|(10000,[360,515,5...|  0.0|
|D'aww! He matches...|    0|0.5530022110246262|[d'aww!, he, matc...|[d'aww!, matches,...|(10000,[233,808,1...|(10000,[233,808,1...|  0.0|
|Hey man, I'm real...|    0|0.5530022110246262|[hey, man,, i'm, ...|[hey, man,, i'm, ...|(10000,[94,222,23...|(10000,[94,222,23...|  0.0|
|"\nMore\nI can't ...|    0|0.

In [16]:
#Logistic regression
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0,  weightCol=WEIGHT_COL)
lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("probability","label","prediction") \
    .orderBy("probability", ascending=True) \
    .show(n = 10, truncate = 30)

evaluator = BinaryClassificationEvaluator(labelCol="label")
evaluator.evaluate(predictions)

22/12/17 20:11:17 WARN TaskSetManager: Stage 10 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:11:27 WARN TaskSetManager: Stage 11 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:11:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/17 20:11:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/12/17 20:11:35 WARN TaskSetManager: Stage 12 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:11:36 WARN TaskSetManager: Stage 13 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:11:36 WARN TaskSetManager: Stage 14 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:11:36 WARN TaskSetManager: Stage 15 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:11:37 WARN TaskSetManager: Stage 16 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:11:37 WARN TaskSetManager: Stage 17 contains a task of very la

+------------------------------+-----+----------+
|                   probability|label|prediction|
+------------------------------+-----+----------+
|[0.5000192054227558,0.49998...|  0.0|       0.0|
|[0.5000197566921781,0.49998...|  0.0|       0.0|
|[0.5000236014565854,0.49997...|  0.0|       0.0|
|[0.5000364693446994,0.49996...|  0.0|       0.0|
|[0.5000419283038847,0.49995...|  0.0|       0.0|
|[0.5000425957062433,0.49995...|  0.0|       0.0|
|[0.5000567017685917,0.49994...|  0.0|       0.0|
|[0.5000700948718092,0.49992...|  0.0|       0.0|
|[0.5000756643575277,0.49992...|  0.0|       0.0|
|[0.5000798842781096,0.49992...|  0.0|       0.0|
+------------------------------+-----+----------+
only showing top 10 rows

22/12/17 20:11:52 WARN TaskSetManager: Stage 35 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


0.8645312033103555

In [17]:
#Naive Bayes
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(smoothing=1,weightCol=WEIGHT_COL)
model = nb.fit(trainingData)

predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("probability","label","prediction") \
    .orderBy("probability", ascending=True) \
    .show(n = 10, truncate = 30)
evaluator = BinaryClassificationEvaluator(labelCol="label")
evaluator.evaluate(predictions)

22/12/17 20:12:03 WARN TaskSetManager: Stage 46 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:12:11 WARN TaskSetManager: Stage 49 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------+-----+----------+
|                   probability|label|prediction|
+------------------------------+-----+----------+
|[0.5004322041815917,0.49956...|  1.0|       0.0|
|[0.5005617099837624,0.49943...|  1.0|       0.0|
|[0.5010980064229973,0.49890...|  0.0|       0.0|
|[0.5014217020508416,0.49857...|  1.0|       0.0|
|[0.5029956574492451,0.49700...|  0.0|       0.0|
|[0.5031244113362693,0.49687...|  0.0|       0.0|
|[0.5042498797438314,0.49575...|  0.0|       0.0|
|[0.5044206213355158,0.49557...|  0.0|       0.0|
|[0.5046222122938804,0.49537...|  0.0|       0.0|
|[0.5048446067230835,0.49515...|  0.0|       0.0|
+------------------------------+-----+----------+
only showing top 10 rows

22/12/17 20:12:19 WARN TaskSetManager: Stage 50 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


0.6145252040900693

In [18]:
#random forest 
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32,
                           weightCol=WEIGHT_COL)

rfModel = rf.fit(trainingData)

predictions = rfModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("probability","label","prediction") \
    .orderBy("probability", ascending=True) \
    .show(n = 10, truncate = 30)
evaluator = BinaryClassificationEvaluator(labelCol="label")
evaluator.evaluate(predictions)

22/12/17 20:12:28 WARN TaskSetManager: Stage 61 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:12:32 WARN TaskSetManager: Stage 62 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:12:38 WARN TaskSetManager: Stage 63 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:12:50 WARN TaskSetManager: Stage 65 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:13:15 WARN TaskSetManager: Stage 67 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:13:22 WARN TaskSetManager: Stage 69 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:13:31 WARN DAGScheduler: Broadcasting large task binary with size 1006.7 KiB
22/12/17 20:13:31 WARN TaskSetManager: Stage 71 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:13:40 WARN TaskSetManager: Stage 73 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------+-----+----------+
|                   probability|label|prediction|
+------------------------------+-----+----------+
|[0.5000002724831206,0.49999...|  1.0|       0.0|
|[0.5000012924778408,0.49999...|  0.0|       0.0|
|[0.5000051576074206,0.49999...|  0.0|       0.0|
|[0.5000072433375797,0.49999...|  0.0|       0.0|
|[0.5000075392424492,0.49999...|  0.0|       0.0|
|[0.5000113873737542,0.49998...|  0.0|       0.0|
|[0.5000115788588114,0.49998...|  0.0|       0.0|
|[0.5000120916912075,0.49998...|  0.0|       0.0|
|[0.5000120916912075,0.49998...|  0.0|       0.0|
|[0.5000120916912075,0.49998...|  0.0|       0.0|
+------------------------------+-----+----------+
only showing top 10 rows

22/12/17 20:13:48 WARN TaskSetManager: Stage 74 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


0.8218286373489716

Выводы - несбалансированность классов хорошо пофиксить не удалось, хотя на это ушла куча времени.
numFeatures - интересный параметр, маленькими значениями получает нелицеприятные метрики, большими - переобучение и большое число измерений(много памяти ест). 

In [19]:
#word2vec
from pyspark.ml.feature import Word2Vec
w2v = Word2Vec(vectorSize=2, minCount=0, inputCol=remover.getOutputCol(), outputCol="features")
pipeline = Pipeline() \
.setStages([tokenizer,remover,w2v,label_stringIdx])

In [21]:
pipelineFit1 = pipeline.fit(df)
dataset1 = pipelineFit1.transform(df)
dataset1.show()

22/12/17 20:15:54 WARN TaskSetManager: Stage 90 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:16:09 WARN TaskSetManager: Stage 92 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:20:09 WARN TaskSetManager: Stage 95 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:20:15 WARN TaskSetManager: Stage 98 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+-----+------------------+--------------------+--------------------+--------------------+-----+
|        comment_text|toxic|            weight|      text_tokenized|              result|            features|label|
+--------------------+-----+------------------+--------------------+--------------------+--------------------+-----+
|Explanation\nWhy ...|    0|0.5530022110246262|[explanation, why...|[explanation, edi...|[-0.1200168296694...|  0.0|
|D'aww! He matches...|    0|0.5530022110246262|[d'aww!, he, matc...|[d'aww!, matches,...|[-0.0564699582755...|  0.0|
|Hey man, I'm real...|    0|0.5530022110246262|[hey, man,, i'm, ...|[hey, man,, i'm, ...|[-0.1277278564362...|  0.0|
|"\nMore\nI can't ...|    0|0.5530022110246262|[", more, i, can'...|[can't, make, rea...|[-0.0710091752160...|  0.0|
|You, sir, are my ...|    0|0.55300221

In [22]:
#Logistic regression
(trainingData, testData) = dataset1.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0,  weightCol=WEIGHT_COL)
lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("probability","label","prediction") \
    .orderBy("probability", ascending=True) \
    .show(n = 10, truncate = 30)

evaluator = BinaryClassificationEvaluator(labelCol="label")
evaluator.evaluate(predictions)

22/12/17 20:22:12 WARN TaskSetManager: Stage 99 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:22:17 WARN TaskSetManager: Stage 100 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


22/12/17 20:22:21 WARN TaskSetManager: Stage 101 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:22:22 WARN TaskSetManager: Stage 102 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:22:22 WARN TaskSetManager: Stage 103 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:22:22 WARN TaskSetManager: Stage 104 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:22:22 WARN TaskSetManager: Stage 105 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:22:23 WARN TaskSetManager: Stage 106 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.
22/12/17 20:22:23 WARN TaskSetManager: Stage 107 contains a task of very large size (18323 KiB). The maximum recommended task size is 10

+------------------------------+-----+----------+
|                   probability|label|prediction|
+------------------------------+-----+----------+
|[0.5000020710300407,0.49999...|  0.0|       0.0|
|[0.5000165011282081,0.49998...|  0.0|       0.0|
|[0.5000167989511154,0.49998...|  0.0|       0.0|
|[0.5000170098357869,0.49998...|  0.0|       0.0|
|[0.5000258160825715,0.49997...|  0.0|       0.0|
|[0.5000306210659297,0.49996...|  1.0|       0.0|
|[0.5000360747080971,0.49996...|  0.0|       0.0|
|[0.5000386643207583,0.49996...|  0.0|       0.0|
|[0.5000414889130453,0.49995...|  0.0|       0.0|
|[0.5000419502242218,0.49995...|  0.0|       0.0|
+------------------------------+-----+----------+
only showing top 10 rows

22/12/17 20:22:29 WARN TaskSetManager: Stage 109 contains a task of very large size (18323 KiB). The maximum recommended task size is 1000 KiB.


0.8038256298681676

Выводы для Word2vec - очень долго рендерится( лучше юзать предобученное?)),но даже на двух относительно грустных метрик показывает не настолькьо грустные результаты. В принципе может показывать результаты лучше, чем предыдущие алгоритмы, но работать будет яяяявно дольше